In [2]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import time

pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment',None)

In [3]:
headers = {
   'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
   'Accept-Encoding' : 'gzip, deflate, br',
   'Accept-Language' : 'en-US,en;q=0.5',
   'cache-control' : 'no-cache',
   'content-encoding' : 'gzip',
   'Cookie' : 'x-amz-continuous-deployment-state=AYABeNfmFSgaeHo%2FLtR+c8h+1+QAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADFr4P9HOJU4H5lQbnQAw7AEiGQ0HM1Fk4U7zntBSGv2xUhbmywBg40MJxKPDYY62t9qinQgjNGOrqkxhwHasAgAAAAAMAAQAAAAAAAAAAAAAAAAAAMafJVan5wcqx9P0nZC5Yc7%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwxkuqjvKJA7NJe6hGuniFPlxg9rVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3p; search=6|1708819459747%7…AAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADFr4P9HOJU4H5lQbnQAw7AEiGQ0HM1Fk4U7zntBSGv2xUhbmywBg40MJxKPDYY62t9qinQgjNGOrqkxhwHasAgAAAAAMAAQAAAAAAAAAAAAAAAAAAMafJVan5wcqx9P0nZC5Yc7%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwxkuqjvKJA7NJe6hGuniFPlxg9rVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3p; JSESSIONID=F41F9EA33C943545A252BC4A41C32C2D',
   'Sec-Fetch-Dest' : 'document',
   'Sec-Fetch-Mode' : 'navigate',
   'Sec-Fetch-Site' : 'same-origin',
   'Sec-Fetch-User' : '?1',
   'Upgrade-Insecure-Requests' : '1',
   'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; rv:109.0) Gecko/20100101 Firefox/116.0'

}

In [4]:
# get around the response outputting JSON and getting an error from structure, manually build a new response list
# write loops to create lists for each page request 
# useful for scraping through more than a single page


with requests.session() as s:
    city = 'santa-barbara-county-ca/'
    page = 1
    end_page = 10
    url = ''
    url_list = []
    
    while page <= end_page:
        url = 'https://www.zillow.com/' +city+'/rentals/' f'{page}_p/'
        url_list.append(url)
        page += 1
    
    request = ''
    request_list = []
    
    for url in url_list:
        time.sleep(2)
        request = s.get(url, headers=headers)
        request_list.append(request)
    
soup = ''
soup_list = []

# loops through the request list in order to fill the BeautifulSoup list
for request in request_list:
    soup = BeautifulSoup(request.content, 'html.parser')
    soup_list.append(soup)

In [7]:
#current error: the elements scraped for beds does not match with prices or the total amount of properties scraped

df_list = []
for soup in soup_list:
    df = pd.DataFrame()
    for i in soup:
        address = soup.find_all (class_= 'StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 jnnxAW property-card-link')
        price = list(soup.find_all (class_='PropertyCardWrapper__StyledPriceGridContainer-srp__sc-16e8gqd-0 kSsByo'))
        #try:
            #beds = soup.find_all("ul", class_='StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID')
        #except AttributeError:
        beds = soup.find_all("ul", class_='StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 dbDWjx')
        
        df['price'] = price
        df['address'] = address
        #df['beds'] = beds

    df_list.append(df)
    

In [70]:
df_list = []
for soup in soup_list:
    df = pd.DataFrame()
    for i in soup:
        beds = soup.find_all("ul", class_='StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID')
        df['beds'] = beds 
    
print(df['beds'][7])

<ul class="StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID"><li><b>3</b> <abbr>bds</abbr></li><li><b>2</b> <abbr>ba</abbr></li><li><b>1,700</b> <abbr>sqft</abbr></li></ul>


In [8]:
df_list[0]



,price,address
0,"[[$4,220+ 1 bd]]","[[Arrive Los Carneros II | 6720 Calle Koral, G..."
1,"[[$2,956/mo]]","[[Pacific Oaks, 7170 Davenport Rd #7260-105, G..."
2,"[[$2,224+ 1 bd]]","[[Hancock Terrace Apartments | 534 E Boone St,..."
3,"[[$2,837+ 2 bds]]","[[St Claire | 1735 Biscayne St, Santa Maria, CA]]"
4,"[[$3,575+ 1 bd]]","[[Cortona Point Apartments | 100 Cortona Dr, G..."
5,"[[$2,469+ 2 bds]]","[[Siena Apartments | 1245 W Sonya Ln, Santa Ma..."
6,"[[$3,675+/mo]]","[[The Tahitian Apartments, 6739 El Colegio Rd,..."
7,"[[$2,895+ 2 bds]]","[[Montiavo | 2460 Rubel Way, Santa Maria, CA]]"
8,"[[$3,275+ 3 bds]]","[[Montiavo | 2460 S Rubel Way, Santa Maria, CA]]"
